<a href="https://colab.research.google.com/github/rushikeshnakhate/A2DataTech/blob/master/Stocks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [289]:
# pip install jugaad_data

In [290]:
from pathlib import Path
from datetime import date
from jugaad_data.nse import NSELive
from jugaad_data.nse import bhavcopy_save
import pandas as pd
output_dir = Path("/content/sample_data")
import os

In [291]:
import pickle
def load_from_pickle(file_path):
    try:
      new_file_path = file_path.with_suffix(".pkl")
      with open(new_file_path,'rb') as file:
        return pickle.load(file)
    except FileNotFoundError:
      return None

def save_to_pickle(obj_to_be_pkl):
  pkl_file_name = obj_to_be_pkl.with_suffix(".pkl")
  with open(pkl_file_name,'wb') as file:
    pickle.dump(obj_to_be_pkl,file)

In [292]:
def download_data(start_date, output_file ,download_function):
    data = load_from_pickle(output_file)
    if data is None:
        generated_file = download_function(start_date, output_file)
        os.rename(generated_file,output_file)
        save_to_pickle(output_file)
    return load_from_pickle(output_file)

In [293]:
def download_bhavcopy(start_date,output_file):
    generated_filename = bhavcopy_save(start_date, output_dir)
    return generated_filename

In [294]:
def fetch_or_genenate_stocks(start_date):
  output_file  = Path(output_dir)/("NseData_" + str(start_date) +".csv")
  return download_data(start_date, output_file,download_bhavcopy)

In [295]:
def print_df(df):
  from google.colab import data_table
  data_table.enable_dataframe_formatter()
  display(df)

In [296]:
import yfinance as yf

def get_sector(tick : str = ""):
  tick_ = tick + ".NS"
  try:
    ticker = yf.Ticker(tick_)
    sector = ticker.info['sector']
    return sector
  except Exception as e:
    return "NOT_FOUND_ON_YAHOO"

In [322]:
import concurrent.futures

class DataWithSector:
  def __init__(self,start_date):
    self.stock_with_sector_output_file  = Path(output_dir)/("NseDataWithSector_" + str(start_date) +".csv")

  # Function to get sector in parallel
  def add_sector_column_to_stock_data_concurently(self,df):
    with concurrent.futures.ThreadPoolExecutor() as executor:
      df['SECTOR'] = list(executor.map(get_sector, df['SYMBOL']))
    return df


  def fetch_or_genenate_stocks_with_sector(self):

      data = load_from_pickle(self.stock_with_sector_output_file)
      if data is None:
        #  call step to get stocks
        stock_data = fetch_or_genenate_stocks(self.start_date)
        stock_data_df = pd.read_csv(stock_data)
        data = self.add_sector_column_to_stock_data_concurently(stock_data_df)
        data.to_csv(self.stock_with_sector_output_file)
        save_to_pickle(self.stock_with_sector_output_file)

      return load_from_pickle(self.stock_with_sector_output_file)

  def get_stock_with_sector_df(self):
    return pd.read_csv(self.stock_with_sector_output_file)

In [323]:
from abc import ABC, abstractmethod

class Performace(ABC):
  def __init__(self, start_date):
    self.start_date = start_date

  @abstractmethod
  def calculate_performace(self,start_date):
    pass

class PerformaceByMeanAndDaviation(Performace):
  def __init__(self, start_date):
    super().__init__(start_date)
    self.sector_performance_df = pd.DataFrame()
    self.sector_performance_output_file = Path(output_dir)/("SectorPeformance_" + str(self.start_date) +".csv")

  def calculate_performace(self,stocks_with_sector_df):
      # Calculate daily returns
      stocks_with_sector_df['DAILY_RETURN'] = (stocks_with_sector_df['CLOSE'] - stocks_with_sector_df['OPEN']) / stocks_with_sector_df['OPEN']

      # Group by 'SECTOR' and calculate average daily return and volatility
      self.sector_performance_df = stocks_with_sector_df.groupby('SECTOR').agg({
          # stocks_list=('STOCK', lambda x: list(x))
          'DAILY_RETURN': ['std','mean','count']
      }).reset_index()

      self.sector_performance_df["TRADEDATE"] = self.start_date

      # Flatten the column names, but only for columns where the second part is not an empty string
      self.sector_performance_df.columns = [
          f"{col[0]}_{col[1]}" if col[1] else col[0] for col in self.sector_performance_df.columns
      ]
      return self.sector_performance_df

  def fetch_or_populate_performace(self):
    data = load_from_pickle(self.sector_performance_output_file)
    if data is not None:
      self.sector_performance_df  = pd.read_csv(self.sector_performance_output_file)
      return data

    # call step to get stocks , get sector and merge them
    dataWithSector = DataWithSector (self.start_date)
    stock_with_sector = dataWithSector.fetch_or_genenate_stocks_with_sector()
    stocks_with_sector_df = pd.read_csv(stock_with_sector)

    self.sector_performance_df = self.calculate_performace(stocks_with_sector_df)
    self.sector_performance_df.to_csv(self.sector_performance_output_file,index=False)
    save_to_pickle(self.sector_performance_output_file)

    data =  load_from_pickle(self.sector_performance_output_file)
    self.sector_performance_df  = pd.read_csv(self.sector_performance_output_file)
    return data

  def get_df(self):
    return self.sector_performance_df



In [324]:
import matplotlib.pyplot as plt
import pandas as pd

# Assuming your data is stored in a DataFrame named 'df'
# If not, you need to create a DataFrame with the provided data.

def plot(df):
  fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 8))

  df.plot(kind='bar', x='SECTOR', y='DAILY_RETURN_mean', ax=ax1, legend=False, color='green')
  ax1.set_ylabel('Mean Daily Return')
  ax1.set_title('Mean Daily Return for Different Sectors')

  df.plot(kind='bar', x='SECTOR', y='DAILY_RETURN_std', ax=ax2, legend=False, color='blue')
  ax2.set_ylabel('Standard Deviation of Daily Return')
  ax2.set_title('Standard Deviation of Daily Return for Different Sectors')

  plt.tight_layout()
  plt.show()





In [331]:
if __name__ == "__main__":
  start_date = date(2024, 1, 17)
  # Step 1
  performaceByMeanAndDaviation = PerformaceByMeanAndDaviation(start_date)
  performaceByMeanAndDaviation.fetch_or_populate_performace()
  print_df(performaceByMeanAndDaviation.get_df())
  # plot(performaceByMeanAndDaviation.get_df())

  dataWithSector = DataWithSector(start_date)
  df = dataWithSector.get_stock_with_sector_df()

  grouped_df = df.groupby('SECTOR')
  for group_name, group_data in grouped_df:
    print_df(group_name)
    print_df(group_data)



,SECTOR,DAILY_RETURN_std,DAILY_RETURN_mean,DAILY_RETURN_count,TRADEDATE
0,Basic Materials,0.022388,-0.005030,285,2024-01-17
1,Communication Services,0.042719,0.009696,55,2024-01-17
2,Consumer Cyclical,0.022795,-0.004585,311,2024-01-17
3,Consumer Defensive,0.026542,-0.000234,115,2024-01-17
4,Energy,0.014375,0.000584,29,2024-01-17
5,Financial Services,0.025182,-0.004131,174,2024-01-17
6,Healthcare,0.016941,-0.004742,122,2024-01-17
7,Industrials,0.022092,0.001383,315,2024-01-17
8,NOT_FOUND_ON_YAHOO,0.019691,-0.007520,204,2024-01-17
9,Real Estate,0.029215,0.005467,48,2024-01-17


'Basic Materials'

,Unnamed: 0,SYMBOL,SERIES,OPEN,HIGH,LOW,CLOSE,LAST,PREVCLOSE,TOTTRDQTY,TOTTRDVAL,TIMESTAMP,TOTALTRADES,ISIN,Unnamed: 13,SECTOR
11,11,20MICRONS,EQ,165.00,174.45,162.60,169.70,170.0,165.55,232724,3.963082e+07,17-JAN-2024,13212,INE144J01027,NaN,Basic Materials
81,81,AARTIIND,EQ,594.95,600.75,580.15,583.05,583.2,600.25,1677162,9.893850e+08,17-JAN-2024,46222,INE769A01020,NaN,Basic Materials
83,83,AARTISURF,EQ,690.00,708.00,685.30,701.70,703.0,697.05,43232,3.032538e+07,17-JAN-2024,7513,INE09EO01013,NaN,Basic Materials
84,84,AARTISURF,P1,193.50,193.50,193.50,193.50,193.5,230.00,5,9.675000e+02,17-JAN-2024,1,INE09EO04017,NaN,Basic Materials
96,96,ABMINTLLTD,BE,55.05,55.05,55.00,55.00,55.0,55.05,499,2.745035e+04,17-JAN-2024,11,INE251C01025,NaN,Basic Materials
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2606,2606,WSTCSTPAPR,EQ,735.00,736.65,721.20,725.75,726.0,740.85,165403,1.205317e+08,17-JAN-2024,12801,INE976A01021,NaN,Basic Materials
2609,2609,XPROINDIA,EQ,1120.00,1155.00,1110.50,1129.00,1129.0,1125.30,52211,5.896550e+07,17-JAN-2024,7331,INE445C01015,NaN,Basic Materials
2611,2611,YASHO,EQ,1629.55,1650.90,1629.55,1641.60,1645.0,1629.55,13928,2.286747e+07,17-JAN-2024,1984,INE616Z01012,NaN,Basic Materials
2624,2624,ZENITHSTL,BE,7.00,7.00,7.00,7.00,7.0,6.95,79513,5.565910e+05,17-JAN-2024,151,INE318D01020,NaN,Basic Materials


'Communication Services'

,Unnamed: 0,SYMBOL,SERIES,OPEN,HIGH,LOW,CLOSE,LAST,PREVCLOSE,TOTTRDQTY,TOTTRDVAL,TIMESTAMP,TOTALTRADES,ISIN,Unnamed: 13,SECTOR
126,126,AFFLE,EQ,1289.00,1314.75,1270.00,1285.70,1288.00,1290.05,186814,2.419567e+08,17-JAN-2024,22497,INE00WC01027,NaN,Communication Services
299,299,BAGFILMS,BE,12.85,13.35,12.75,12.80,12.90,13.15,321696,4.177087e+06,17-JAN-2024,1104,INE116D01028,NaN,Communication Services
311,311,BALAJITELE,EQ,91.00,94.05,88.00,91.50,91.40,92.35,1048951,9.681932e+07,17-JAN-2024,7797,INE794B01026,NaN,Communication Services
343,343,BCG,EQ,19.85,20.00,19.55,19.70,19.75,20.00,19685771,3.891224e+08,17-JAN-2024,20963,INE425B01027,NaN,Communication Services
368,368,BHARTIARTL,EQ,1092.00,1108.95,1078.80,1085.25,1081.10,1095.90,4568700,4.983594e+09,17-JAN-2024,170889,INE397D01024,NaN,Communication Services
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2468,2468,UFO,EQ,129.45,143.80,127.70,137.30,137.80,130.30,1748670,2.417088e+08,17-JAN-2024,24588,INE527H01019,NaN,Communication Services
2484,2484,UNIINFO,BE,30.80,30.80,29.35,29.95,30.60,30.80,34294,1.024719e+06,17-JAN-2024,121,INE481Z01011,NaN,Communication Services
2533,2533,VERTOZ,EQ,610.00,689.55,607.00,689.55,689.55,626.90,757182,5.045756e+08,17-JAN-2024,39899,INE188Y01015,NaN,Communication Services
2620,2620,ZEEL,EQ,248.60,257.45,245.00,245.85,245.50,253.30,19673798,4.943111e+09,17-JAN-2024,109120,INE256A01028,NaN,Communication Services


...

,Unnamed: 0,SYMBOL,SERIES,OPEN,HIGH,LOW,CLOSE,LAST,PREVCLOSE,TOTTRDQTY,TOTTRDVAL,TIMESTAMP,TOTALTRADES,ISIN,Unnamed: 13,SECTOR
145,145,AJMERA,EQ,503.85,515.85,498.00,505.15,509.45,507.10,225081,1.141040e+08,17-JAN-2024,10248,INE298G01027,NaN,Real Estate
157,157,ALEMBICLTD,EQ,94.00,96.00,93.00,94.05,94.35,95.10,1529380,1.441379e+08,17-JAN-2024,9090,INE426A01027,NaN,Real Estate
180,180,AMJLAND,EQ,40.10,40.50,38.85,39.45,39.70,40.30,142771,5.645094e+06,17-JAN-2024,1224,INE606A01024,NaN,Real Estate
184,184,ANANTRAJ,EQ,315.90,318.95,310.15,316.25,318.35,319.85,1142075,3.609865e+08,17-JAN-2024,16397,INE242C01024,NaN,Real Estate
218,218,ARIHANTSUP,EQ,324.25,330.35,313.70,318.30,320.85,329.20,130427,4.171370e+07,17-JAN-2024,8098,INE643K01018,NaN,Real Estate
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2384,2384,TECHIN,BE,28.05,28.05,27.50,28.05,28.05,26.75,24461,6.854661e+05,17-JAN-2024,91,INE778A01021,NaN,Real Estate
2392,2392,TEXINFRA,EQ,106.00,110.00,105.00,107.00,106.50,107.90,580988,6.256811e+07,17-JAN-2024,7162,INE435C01024,NaN,Real Estate
2439,2439,TREL,EQ,51.20,52.90,49.95,50.60,51.05,51.25,1009907,5.189000e+07,17-JAN-2024,4332,INE0O3901029,NaN,Real Estate
2487,2487,UNITECH,BZ,10.95,11.90,10.85,11.90,11.90,11.35,18716961,2.132190e+08,17-JAN-2024,12905,INE694A01020,NaN,Real Estate


'Technology'

,Unnamed: 0,SYMBOL,SERIES,OPEN,HIGH,LOW,CLOSE,LAST,PREVCLOSE,TOTTRDQTY,TOTTRDVAL,TIMESTAMP,TOTALTRADES,ISIN,Unnamed: 13,SECTOR
22,22,3IINFOLTD,EQ,53.05,55.45,52.70,53.60,53.50,54.20,1907358,1.034260e+08,17-JAN-2024,9584,INE748C01038,NaN,Technology
29,29,63MOONS,BE,440.05,448.25,436.00,438.90,441.95,448.25,73571,3.248021e+07,17-JAN-2024,1752,INE111B01023,NaN,Technology
75,75,AAATECH,BE,90.20,99.60,90.20,97.60,97.85,94.95,50195,4.888005e+06,17-JAN-2024,443,INE0D0U01013,NaN,Technology
102,102,ACCELYA,EQ,1830.00,1869.70,1795.00,1837.35,1843.00,1875.40,52104,9.557722e+07,17-JAN-2024,5162,INE793A01012,NaN,Technology
119,119,ADROITINFO,BE,29.10,29.95,28.95,29.95,29.95,28.55,422247,1.252532e+07,17-JAN-2024,1367,INE737B01033,NaN,Technology
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2600,2600,WIPRO,EQ,475.15,491.50,472.00,482.40,483.00,485.00,14334466,6.926541e+09,17-JAN-2024,178223,INE075A01022,NaN,Technology
2607,2607,XCHANGING,EQ,112.75,115.70,110.00,112.80,113.25,114.30,749805,8.511428e+07,17-JAN-2024,8722,INE692G01013,NaN,Technology
2608,2608,XELPMOC,EQ,130.75,140.95,128.50,130.95,130.50,132.95,168798,2.264920e+07,17-JAN-2024,2998,INE01P501012,NaN,Technology
2618,2618,ZAGGLE,EQ,213.00,217.85,207.40,208.30,208.00,216.85,1079865,2.288826e+08,17-JAN-2024,17719,INE07K301024,NaN,Technology


'Utilities'

,Unnamed: 0,SYMBOL,SERIES,OPEN,HIGH,LOW,CLOSE,LAST,PREVCLOSE,TOTTRDQTY,TOTTRDVAL,TIMESTAMP,TOTALTRADES,ISIN,Unnamed: 13,SECTOR
111,111,ADANIENSOL,EQ,1124.95,1128.75,1082.45,1087.15,1088.00,1126.35,1456313,1.598512e+09,17-JAN-2024,72307,INE931S01010,NaN,Utilities
113,113,ADANIGREEN,EQ,1653.00,1656.95,1605.10,1615.00,1613.90,1666.80,1151828,1.878212e+09,17-JAN-2024,60882,INE364U01010,NaN,Utilities
115,115,ADANIPOWER,BE,529.45,529.90,506.10,521.45,521.85,532.30,1806770,9.447921e+08,17-JAN-2024,34011,INE814H01011,NaN,Utilities
257,257,ATGL,EQ,1012.85,1019.45,1000.00,1004.10,1007.00,1026.05,1250880,1.259596e+09,17-JAN-2024,58401,INE399L01023,NaN,Utilities
358,358,BFUTILITIE,EQ,589.00,607.00,582.75,586.50,586.00,593.10,171999,1.020067e+08,17-JAN-2024,4072,INE243D01012,NaN,Utilities
454,454,CESC,EQ,137.95,141.40,136.15,138.35,137.95,138.85,7388183,1.028333e+09,17-JAN-2024,48060,INE486A01021,NaN,Utilities
611,611,DPSCLTD,EQ,19.00,19.10,18.80,18.90,19.05,19.10,542792,1.029366e+07,17-JAN-2024,2260,INE360C01024,NaN,Utilities
681,681,ENERGYDEV,BE,26.05,26.05,25.00,25.20,25.60,26.15,111796,2.840848e+06,17-JAN-2024,579,INE306C01019,NaN,Utilities
749,749,GAIL,EQ,165.50,169.00,163.60,164.10,164.65,166.90,31113706,5.173199e+09,17-JAN-2024,111281,INE129A01019,NaN,Utilities
785,785,GIPCL,EQ,176.85,184.30,176.20,176.90,176.50,178.40,1140313,2.056033e+08,17-JAN-2024,17821,INE162A01010,NaN,Utilities
